# 債券利率蒙地卡羅模擬教學

本 Jupyter Notebook 旨在教學演示如何使用 Python 進行債券利率的蒙地卡羅模擬。
程式碼中的變數名稱、公式與註解都對標 'equation.md' 文件，以利於學習與對照。

## 內容包含：

1.  **四種利率模型**：
    *   **Vasicek 模型**：用於模擬無風險利率 (drf)，具有均值回歸特性。
    *   **Cox-Ingersoll-Ross (CIR) 模型**：Vasicek 的變體，確保利率為正。
    *   **幾何布朗運動 (GBM) 模型**：一個常用於股價的簡化利率模型。
    *   **有風險利率模型**：在無風險利率上疊加一個隨機的信用利差。
2.  **債券價格近似計算**：
    *   根據模擬出的利率路徑，使用「修正存續期間」與「價格曲度」來近似計算債券價格的變化。

## 1. 載入所需函式庫

首先，我們載入 `numpy` 用於數值計算，`matplotlib.pyplot` 用於繪圖，以及 `datetime` 用於處理日期。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
try:
  # Colab 進行matplotlib繪圖時顯示繁體中文
  # 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
  import requests
  url = "https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download"
  resp = requests.get(url)
  resp.raise_for_status()
  with open("TaipeiSansTCBeta-Regular.ttf", "wb") as f:
      f.write(resp.content)
  import matplotlib
  matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
  matplotlib.rc('font', family='Taipei Sans TC Beta')
except:
  # 如果你是在本地端執行
  plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei',]

## 2. 利率模擬函數

此區塊定義了四種不同的利率模型函數，用於生成未來的利率路徑。

### 2.1 Vasicek 模型

**隨機微分方程 (SDE):** `dr_t = k(θ - r_t)dt + σdW_t`

此模型描述了一個具有「均值回歸」特性的利率。利率 `r_t` 會傾向於回歸到其長期平均水平 `θ`，回歸的速度由 `k` 控制，而 `σ` 則代表其隨機波動的幅度。

In [ ]:
def simulate_vasicek(r_0, k, theta, sigma, T, dt, n_simulations, dW_t):
    """
    使用 Vasicek 模型模擬利率路徑。
    
    :param r_0: float, 初始利率 (r_t 在 t=0 的值)。
    :param k: float, 均值回歸速度 (kappa)，代表利率回復到長期均值的速度。
    :param theta: float, 長期平均利率或均衡水平 (θ)。
    :param sigma: float, 波動率 (σ)，代表利率隨機波動的幅度。
    :param T: float, 總模擬時長(年)。
    :param dt: float, 每個時間步長(年)。
    :param n_simulations: int, 模擬路徑的數量。
    :param dW_t: np.ndarray, 預先生成的維納過程增量 (dW_t)，代表隨機衝擊。
    :return: np.ndarray, 模擬的利率路徑。
    """
    num_steps = int(T / dt)
    r_paths = np.zeros((n_simulations, num_steps + 1))
    r_paths[:, 0] = r_0

    for t in range(1, num_steps + 1):
        r_paths[:, t] = r_paths[:, t-1] + k * (theta - r_paths[:, t-1]) * dt + sigma * dW_t[:, t-1]
        r_paths[:, t] = np.maximum(0.0001, r_paths[:, t]) # 確保利率為正
        
    return r_paths

### 2.2 Cox-Ingersoll-Ross (CIR) 模型

**隨機微分方程 (SDE):** `dr_t = k(θ - r_t)dt + σ√r_t dW_t`

CIR 模型是 Vasicek 的變體，其主要優點是能確保利率恆為正值（在 `2kθ > σ²` 的條件下）。波動項 `σ√r_t` 表示利率越高，其波動也越大。

In [ ]:
def simulate_cir(r_0, k, theta, sigma, T, dt, n_simulations, dW_t):
    """
    使用 Cox-Ingersoll-Ross (CIR) 模型模擬利率路徑。
    
    :param r_0: float, 初始利率。
    :param k: float, 均值回歸速度。
    :param theta: float, 長期平均利率。
    :param sigma: float, 波動率。
    :param T: float, 總模擬時長(年)。
    :param dt: float, 每個時間步長(年)。
    :param n_simulations: int, 模擬路徑的數量。
    :param dW_t: np.ndarray, 預先生成的維納過程增量。
    :return: np.ndarray, 模擬的利率路徑。
    """
    num_steps = int(T / dt)
    r_paths = np.zeros((n_simulations, num_steps + 1))
    r_paths[:, 0] = r_0

    for t in range(1, num_steps + 1):
        sqrt_r = np.sqrt(np.maximum(0, r_paths[:, t-1]))
        r_paths[:, t] = r_paths[:, t-1] + k * (theta - r_paths[:, t-1]) * dt + sigma * sqrt_r * dW_t[:, t-1]
        r_paths[:, t] = np.maximum(0.0001, r_paths[:, t]) # 再次確保利率為正
        
    return r_paths

### 2.3 幾何布朗運動 (GBM) 模型

**隨機微分方程 (SDE):** `dr_t = μr_t dt + σr_t dW_t`

GBM 是金融領域中一個非常經典的模型，常用於模擬股價。它假設資產的報酬率服從常態分佈。雖然在利率模擬中不如均值回歸模型常用，但作為一個基礎模型仍有其教學價值。

In [ ]:
def simulate_gbm(r_0, mu, sigma, T, dt, n_simulations, dW_t):
    """
    使用幾何布朗運動 (GBM) 模型模擬利率路徑。
    
    :param r_0: float, 初始利率。
    :param mu: float, 長期漂移趨勢。
    :param sigma: float, 年化波動率。
    :param T: float, 總模擬時長(年)。
    :param dt: float, 每個時間步長(年)。
    :param n_simulations: int, 模擬路徑的數量。
    :param dW_t: np.ndarray, 預先生成的維納過程增量。
    :return: np.ndarray, 模擬的利率路徑。
    """
    num_steps = int(T / dt)
    r_paths = np.zeros((n_simulations, num_steps + 1))
    r_paths[:, 0] = r_0

    for t in range(1, num_steps + 1):
        drift = (mu - 0.5 * sigma**2) * dt
        diffusion = sigma * dW_t[:, t-1]
        r_paths[:, t] = r_paths[:, t-1] * np.exp(drift + diffusion)
        r_paths[:, t] = np.maximum(0.0001, r_paths[:, t]) # 確保利率為正
        
    return r_paths

### 2.4 有風險利率模型

**公式:**
1.  `drs_t = drf_t + spread_t`
2.  `d(spread_t) = σ_s dW_s`
3.  `dW_s = ρ * dW_f + √(1 - ρ²) * dZ_t`

此模型在無風險利率 `drf_t` (例如由 Vasicek 模型生成) 的基礎上，疊加一個隨機變動的信用利差 `spread_t`，從而得到有風險利率 `drs_t`。

信用利差的變動 `dW_s` 與無風險利率的變動 `dW_f` 之間存在相關性 `ρ`，這使得模型能更真實地反映市場情況（例如，經濟惡化時，無風險利率可能下降，而信用利差卻會擴大）。

In [ ]:
def simulate_risky_rate(drf_paths, dW_f, initial_spread, sigma_s, rho, T, dt, n_simulations):
    """
    模擬有風險利率 (drs)，即在無風險利率(drf)上疊加信用利差(spread)。
    
    :param drf_paths: np.ndarray, 已模擬好的無風險利率路徑。
    :param dW_f: np.ndarray, 生成 drf_paths 所使用的隨機衝擊。
    :param initial_spread: float, 初始信用利差。
    :param sigma_s: float, 信用利差的波動率。
    :param rho: float, drf 與 spread 變動之間的相關係數。
    :param T: float, 總模擬時長(年)。
    :param dt: float, 每個時間步長(年)。
    :param n_simulations: int, 模擬路徑的數量。
    :return: tuple (np.ndarray, np.ndarray), 分別為模擬的有風險利率路徑和信用利差路徑。
    """
    num_steps = int(T / dt)
    
    # 1. 生成一個獨立的隨機衝擊 dZ_t
    dZ_t = np.random.normal(0, 1, (n_simulations, num_steps)) * np.sqrt(dt)
    
    # 2. 根據相關係數 rho，合成信用利差的隨機衝擊 dW_s
    dW_s = rho * dW_f + np.sqrt(1 - rho**2) * dZ_t
    
    # 3. 模擬信用利差的路徑
    spread_paths = np.zeros((n_simulations, num_steps + 1))
    spread_paths[:, 0] = initial_spread
    for t in range(1, num_steps + 1):
        spread_paths[:, t] = spread_paths[:, t-1] + sigma_s * dW_s[:, t-1]
        spread_paths[:, t] = np.maximum(0.0001, spread_paths[:, t]) # 確保利差為正
        
    # 4. 有風險利率 = 無風險利率 + 信用利差
    drs_paths = drf_paths + spread_paths
    
    return drs_paths, spread_paths

## 3. 債券價格計算函數

在得到利率路徑後，我們需要一個方法來估算債券價格的變化。這裡我們使用基於「修正存續期間 (Modified Duration)」和「曲度 (Convexity)」的二階泰勒展開式來近似計算。

**近似公式:** `ΔP/P ≈ -D_mod * Δy + (C_mod / 2) * (Δy)²`

其中：
-   `ΔP/P` 是價格的變動百分比。
-   `D_mod` 是修正存續期間，衡量價格對利率變動的敏感度（一階）。
-   `C_mod` 是曲度，用於修正存續期間的線性估計誤差（二階）。
-   `Δy` 是殖利率的變化量，我們在此用模擬的利率 `r` 來替代。

**注意：** 在我們的系統中，資料庫提供的 `convT` 值是 `P * C_mod / 2`，因此我們需要先將其轉換回公式中所需的 `C_mod / 2` 項。

In [ ]:
def calculate_price_paths(r_paths, D_mod, convT, initial_price=100.0):
    """
    使用修正存續期間(D_mod)和價格曲度(convT)來近似計算債券價格路徑。
    
    :param r_paths: np.ndarray, 模擬的利率路徑 (y)。
    :param D_mod: float, 債券的修正存續期間 (Modified Duration)。
    :param convT: float, 來自資料庫的價格曲度值 (P * C_mod / 2)。
    :param initial_price: float, 債券的初始價格。
    :return: np.ndarray, 模擬的債券價格路徑。
    """
    num_steps = r_paths.shape[1] - 1
    price_paths = np.zeros_like(r_paths)
    price_paths[:, 0] = initial_price
    
    # 從 convT 反解出公式中需要的 (C_mod / 2) 項
    convexity_factor = convT / initial_price
    
    for t in range(1, num_steps + 1):
        delta_y = r_paths[:, t] - r_paths[:, t-1]
        prev_price = price_paths[:, t-1]
        
        # 計算價格變動百分比 (ΔP/P)
        price_change_percentage = -D_mod * delta_y + convexity_factor * (delta_y ** 2)
        
        # 計算新價格 P(t) = P(t-1) * (1 + ΔP/P)
        price_paths[:, t] = prev_price * (1 + price_change_percentage)
        
    return price_paths

## 4. 主執行區塊

現在我們將前面定義的函數組合起來，進行完整的模擬流程。

### 4.1 設定模擬參數

首先，我們定義債券的基本資料和模擬的通用參數。

In [ ]:
# 債券基本資料 (此處為範例，實際應用中從資料庫讀取)
bond_data = {
    'name': 'CGB10Y',
    'avgYld': 2.2,      # 平均殖利率 (%)
    'mdurT': 8.9,       # 修正存續期間 (D_mod)
    'convT': 45.5,      # 價格曲度 (P * C_mod / 2)
    'maturity': datetime(2034, 5, 15)
}

# 模擬通用參數
n_simulations = 100      # 模擬次數
dt = 1/252               # 時間步長 (年)，假設一年252個交易日

# 計算剩餘到期年限，作為模擬總時長 T
time_to_maturity = (bond_data['maturity'] - datetime.now()).days / 365.25
T = max(time_to_maturity, 0.1) # 確保至少模擬一小段時間

num_steps = int(T / dt)
time_points = np.linspace(0, T, num_steps + 1) # 模擬的時間點數列

# 初始利率 (將百分比轉為小數)
r_0 = bond_data['avgYld'] / 100.0

print(f"債券: {bond_data['name']}")
print(f"剩餘到期年限 (T): {T:.2f} 年")
print(f"模擬次數: {n_simulations}")
print(f"時間步長 (dt): {dt:.4f} 年")
print(f"總步數: {num_steps}")
print(f"初始利率 (r_0): {r_0*100:.2f}%")

### 4.2 設定各模型參數

為每個利率模型設定具體的參數值。這些值通常需要透過歷史數據進行校準。

In [ ]:
# 1. Vasicek & CIR 模型參數 (將百分比轉為小數)
k_vasicek = 0.3
theta_vasicek = 2.2 / 100.0
sigma_vasicek = 0.5 / 100.0

k_cir = 0.2
theta_cir = 2.5 / 100.0
sigma_cir = 0.4 / 100.0

# 2. GBM 模型參數
mu_r = 0.05 / 100.0
sigma_r = 0.6 / 100.0

# 3. 有風險利率模型參數
initial_spread = 0.8 / 100.0
sigma_s = 0.3 / 100.0
rho = 0.6

### 4.3 生成隨機衝擊

為了讓模擬結果可以重現，我們使用 `np.random.seed` 設定隨機種子。然後，為每個需要獨立隨機性的模型預先生成維納過程的增量 `dW_t`。

`dW_t` 是一個服從 `N(0, dt)` 的隨機變數，等價於 `sqrt(dt) * N(0, 1)`。

In [ ]:
np.random.seed(42) # 設定隨機種子以確保結果可重現
dW_vasicek = np.random.normal(0, 1, (n_simulations, num_steps)) * np.sqrt(dt)
dW_cir = np.random.normal(0, 1, (n_simulations, num_steps)) * np.sqrt(dt)
dW_gbm = np.random.normal(0, 1, (n_simulations, num_steps)) * np.sqrt(dt)

### 4.4 執行模擬

呼叫前面定義的函數，生成利率路徑和對應的債券價格路徑。

In [ ]:
print("開始執行利率模擬...")
# 1. 模擬無風險利率 drf (Vasicek)
drf_paths = simulate_vasicek(r_0, k_vasicek, theta_vasicek, sigma_vasicek, T, dt, n_simulations, dW_vasicek)

# 2. 模擬有風險利率 drs (在 drf 基礎上增加信用利差)
drs_paths, spread_paths = simulate_risky_rate(drf_paths, dW_vasicek, initial_spread, sigma_s, rho, T, dt, n_simulations)

# 3. 模擬綜合利率 dr (GBM)
dr_paths = simulate_gbm(r_0, mu_r, sigma_r, T, dt, n_simulations, dW_gbm)

# 4. 模擬 CIR 利率 dr_cir
dr_cir_paths = simulate_cir(r_0, k_cir, theta_cir, sigma_cir, T, dt, n_simulations, dW_cir)
print("利率模擬完成。")

print("開始計算債券價格路徑...")
# 5. 計算對應的債券價格路徑
price_drf_paths = calculate_price_paths(drf_paths, bond_data['mdurT'], bond_data['convT'])
price_drs_paths = calculate_price_paths(drs_paths, bond_data['mdurT'], bond_data['convT'])
price_dr_paths = calculate_price_paths(dr_paths, bond_data['mdurT'], bond_data['convT'])
price_dr_cir_paths = calculate_price_paths(dr_cir_paths, bond_data['mdurT'], bond_data['convT'])
print("價格計算完成。")

## 5. 結果可視化

將模擬結果繪製成圖表，以便直觀地理解利率和價格的動態變化。

### 5.1 利率模擬路徑圖

下圖展示了四種模型生成的前 50 條利率路徑。可以觀察到不同模型的特徵：
- **Vasicek** 和 **CIR** 顯示出向長期均值回歸的趨勢。
- **GBM** 則沒有明顯的均值回歸特性，路徑發散較大。
- **有風險利率** 的路徑整體高於無風險利率，反映了信用利差的存在。

In [ ]:
fig_rates, axes_rates = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
fig_rates.suptitle(f'四種利率模型的模擬路徑 (前 {min(50, n_simulations)} 條)', fontsize=16)

axes_rates[0, 0].plot(time_points, drf_paths[:50, :].T * 100, lw=0.5)
axes_rates[0, 0].set_title('1. 無風險利率 (drf) - Vasicek')
axes_rates[0, 0].set_ylabel('利率 (%)')
axes_rates[0, 0].grid(True, linestyle='--', alpha=0.6)

axes_rates[0, 1].plot(time_points, drs_paths[:50, :].T * 100, lw=0.5)
axes_rates[0, 1].set_title('2. 有風險利率 (drs) - Vasicek + Spread')
axes_rates[0, 1].grid(True, linestyle='--', alpha=0.6)

axes_rates[1, 0].plot(time_points, dr_paths[:50, :].T * 100, lw=0.5)
axes_rates[1, 0].set_title('3. 綜合利率 (dr) - GBM')
axes_rates[1, 0].set_xlabel('時間 (年)')
axes_rates[1, 0].set_ylabel('利率 (%)')
axes_rates[1, 0].grid(True, linestyle='--', alpha=0.6)

axes_rates[1, 1].plot(time_points, dr_cir_paths[:50, :].T * 100, lw=0.5)
axes_rates[1, 1].set_title('4. CIR 利率 (dr_cir)')
axes_rates[1, 1].set_xlabel('時間 (年)')
axes_rates[1, 1].grid(True, linestyle='--', alpha=0.6)

fig_rates.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("tutorial_rate_paths.png")
plt.show()

### 5.2 債券價格模擬路徑圖

下圖展示了與上述利率路徑相對應的債券價格變化。我們可以看到，當利率上升時，債券價格下降，反之亦然，這符合債券的基本特性。

In [ ]:
fig_prices, axes_prices = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
fig_prices.suptitle(f'對應的債券價格模擬路徑 (前 {min(50, n_simulations)} 條)', fontsize=16)

axes_prices[0, 0].plot(time_points, price_drf_paths[:50, :].T, lw=0.5)
axes_prices[0, 0].set_title('基於 drf (Vasicek) 的價格')
axes_prices[0, 0].set_ylabel('債券價格')
axes_prices[0, 0].grid(True, linestyle='--', alpha=0.6)

axes_prices[0, 1].plot(time_points, price_drs_paths[:50, :].T, lw=0.5)
axes_prices[0, 1].set_title('基於 drs (Risky) 的價格')
axes_prices[0, 1].grid(True, linestyle='--', alpha=0.6)

axes_prices[1, 0].plot(time_points, price_dr_paths[:50, :].T, lw=0.5)
axes_prices[1, 0].set_title('基於 dr (GBM) 的價格')
axes_prices[1, 0].set_xlabel('時間 (年)')
axes_prices[1, 0].set_ylabel('債券價格')
axes_prices[1, 0].grid(True, linestyle='--', alpha=0.6)

axes_prices[1, 1].plot(time_points, price_dr_cir_paths[:50, :].T, lw=0.5)
axes_prices[1, 1].set_title('基於 dr_cir (CIR) 的價格')
axes_prices[1, 1].set_xlabel('時間 (年)')
axes_prices[1, 1].grid(True, linestyle='--', alpha=0.6)

fig_prices.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("tutorial_price_paths.png")
plt.show()

### 5.3 最終價格分佈圖

此直方圖顯示了在模擬期結束時，所有模擬路徑的最終債券價格分佈。這對於評估風險至關重要，例如計算 VaR (Value at Risk)。

從圖中可以看出，不同利率模型導致的最終價格分佈形狀和範圍有顯著差異。

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(price_drf_paths[:, -1], bins=50, alpha=0.7, label='基於 drf (Vasicek)', density=True)
plt.hist(price_drs_paths[:, -1], bins=50, alpha=0.7, label='基於 drs (Risky)', density=True)
plt.hist(price_dr_paths[:, -1], bins=50, alpha=0.7, label='基於 dr (GBM)', density=True)
plt.hist(price_dr_cir_paths[:, -1], bins=50, alpha=0.7, label='基於 dr_cir (CIR)', density=True)
plt.title('模擬結束時的債券價格分佈')
plt.xlabel('最終價格')
plt.ylabel('機率密度')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig("tutorial_final_price_dist.png")
plt.show()

## 6. 簡單統計分析

最後，我們對兩種較為重要的模型（無風險和有風險）的最終價格進行統計分析，計算其平均值、標準差和 5% 分位數，並估算 95% VaR。

**VaR (Value at Risk)**：在險價值，衡量在給定的信賴水準（此處為 95%）和持有期間內，預期的最大潛在損失。此處的 VaR 是從初始價格 100 計算的損失。

In [ ]:
final_prices_drf = price_drf_paths[:, -1]
final_prices_drs = price_drs_paths[:, -1]

print("--- 最終價格統計分析 ---")
print(f"模型: 基於 drf (Vasicek)")
print(f"  平均最終價格: {np.mean(final_prices_drf):.4f}")
print(f"  價格標準差: {np.std(final_prices_drf):.4f}")
print(f"  5% 分位數價格: {np.percentile(final_prices_drf, 5):.4f}")
print(f"  95% VaR (從初始價100計算的潛在最大損失): {100 - np.percentile(final_prices_drf, 5):.4f}")

print(f"模型: 基於 drs (Risky)")
print(f"  平均最終價格: {np.mean(final_prices_drs):.4f}")
print(f"  價格標準差: {np.std(final_prices_drs):.4f}")
print(f"  5% 分位數價格: {np.percentile(final_prices_drs, 5):.4f}")
print(f"  95% VaR (從初始價100計算的潛在最大損失): {100 - np.percentile(final_prices_drs, 5):.4f}")
print("------------------------")